In [1]:
import requests
import json
import itertools
import os
from datetime import datetime
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import random
from tqdm import tqdm
import time

# Read tokens from a text file
tokens_file = "./env/tokens.txt"
with open(tokens_file, "r") as file:
    tokens = file.read().splitlines()

# Create an iterator to cycle through the tokens
token_iterator = itertools.cycle(tokens)
current_token = next(token_iterator)

In [2]:
# List of User-Agents for randomization
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
]

# Define headers to authenticate using the first token
headers = {
    "Authorization": f"Bearer {current_token}",
    "User-Agent": random.choice(user_agents),
}

# Setup GraphQL endpoint and client
graphql_url = "https://api.github.com/graphql"
transport = RequestsHTTPTransport(url=graphql_url, headers=headers, use_json=True)
client = Client(transport=transport, fetch_schema_from_transport=True)

In [ ]:
# Test all tokens to verify their validity
def test_all_tokens():
    test_query = gql(
        """
        {
          viewer {
            login
          }
        }
        """
    )
    for i, token in enumerate(tokens):
        headers = {
            "Authorization": f"Bearer {token}",
            "User-Agent": random.choice(user_agents),
        }
        transport = RequestsHTTPTransport(
            url=graphql_url, headers=headers, use_json=True
        )
        client = Client(transport=transport, fetch_schema_from_transport=True)

        try:
            response = client.execute(test_query)
            print(
                f"Token {i+1}/{len(tokens)} is valid. Logged in as: {response['viewer']['login']}"
            )
        except Exception as e:
            print(f"Token {i+1}/{len(tokens)} failed with error: {e}")


# Run the token validation
test_all_tokens()

In [215]:
# Define the GraphQL query
query_template = gql(
    """
    query searchIssues($keyword: String!, $afterCursor: String, $first: Int) {
      search(query: $keyword, type: ISSUE, first: $first, after: $afterCursor) {
        issueCount
        edges {
          cursor
          node {
            ... on PullRequest {
              id
              number
              title
              url
              body
              state
              merged
              createdAt
              updatedAt
              mergeCommit {
                oid
              }
              timeline(last: 50) {
                edges
                {
                  node {
                    __typename
                    ... on LabeledEvent {
                      actor { ... on User { login } }
                      label { ... on Label { name }}
                      createdAt
                    }
                    ... on ClosedEvent { 
                      actor { ... on User { login } }
                      createdAt
                    }
                  }
                }
              }
              commits(first: 50) {
                totalCount
                pageInfo {
                  hasNextPage
                  endCursor
                }
                edges {
                  node {
                    commit {
                      oid
                      message
                      author { ... on GitActor { name } }
                      changedFilesIfAvailable
                      commitUrl
                      committedDate
                    }
                  }
                }
              }
              changedFiles
              headRefName
              baseRefName
              repository {
                nameWithOwner
                stargazerCount
                isFork
              }
              author {
                ... on User {
                  login
                  url
                  repositories {
                    totalCount
                  }
                  contributionsCollection { # it is limited to one year
                    contributionCalendar {
                      totalContributions
                    }
                  }
                  followers {
                    totalCount
                  }
                  following {
                    totalCount
                  }
                }
              }
              labels(first: 10) {
                edges {
                  node {
                    name
                  }
                }
              }
            }
          }
        }
        pageInfo {
          endCursor
          hasNextPage
        }
      }
    }
    """
)

In [216]:
# def get_contributor_count(repo_owner, repo_name):
#     global current_token
#     max_retries = 3
#     retries = 0
#     while retries < max_retries:
#         try:
#             # Randomize User-Agent for each query
#             headers["User-Agent"] = random.choice(user_agents)
#             headers["Authorization"] = f"Bearer {current_token}"
#             url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contributors?per_page=1&anon=true"
#             response = requests.get(url, headers=headers)
#             if response.status_code == 200:
#                 return int(response.headers.get("Link", "").split(",")[-1].split("&page=")[-1].split(">")[0]) if "Link" in response.headers else len(response.json())
#             elif response.status_code == 403:
#                 print(f"Rate limit exceeded, switching token... (Attempt {retries + 1}/{max_retries})")
#                 current_token = next(token_iterator)
#                 retries += 1
#             else:
#                 response.raise_for_status()
#         except Exception as e:
#             print(f"Error: {e}, retrying... (Attempt {retries + 1}/{max_retries})")
#             retries += 1
#     raise Exception("Max retries reached. Unable to complete the request.")

In [217]:
transport.headers = headers
# Check rate limit before executing the main query
rate_limit_query = gql(
    """
    query {
      viewer {
        login
      }
      rateLimit {
        limit
        remaining
        used
        resetAt
      }
    }
    """
)
rate_limit_response = client.execute(rate_limit_query)
print(f"Rate limit: {rate_limit_response['rateLimit']}")

Rate limit: {'limit': 5000, 'remaining': 4960, 'used': 40, 'resetAt': '2025-01-17T06:23:34Z'}


In [218]:
rate_limit_query = gql(
    """
    query {
      viewer {
        login
      }
      rateLimit {
        limit
        remaining
        used
        resetAt
      }
    }
    """
)


def execute_query(keyword, first=100, after_cursor=None):
    global current_token
    print(
        f"Executing query with keyword: {keyword}, first: {first}, afterCursor: {after_cursor}"
    )
    while True:
        try:
            # Randomize User-Agent for each query
            headers["User-Agent"] = random.choice(user_agents)
            transport.headers = headers
            # Check rate limit before executing the main query
            rate_limit_response = client.execute(rate_limit_query)
            remaining = rate_limit_response["rateLimit"]["remaining"]
            if remaining < 100:
                print(
                    f"Rate limit remaining ({remaining}) is below threshold. Switching token..."
                )
                # Set up to track whether we have cycled through all tokens
                all_tokens_checked = False
                initial_token = current_token

                while not all_tokens_checked:
                    # Switch to the next token
                    current_token = next(token_iterator)
                    headers["Authorization"] = f"Bearer {current_token}"
                    transport.headers = headers

                    # Check the rate limit of the new token
                    rate_limit_response = client.execute(rate_limit_query)
                    remaining = rate_limit_response["rateLimit"]["remaining"]

                    if remaining >= 100:
                        print(
                            f"Switched to a new token with sufficient rate limit ({remaining} remaining)."
                        )
                        break

                    # Check if we have cycled through all tokens
                    if current_token == initial_token:
                        print("All tokens are below threshold. Waiting for 1 hour...")
                        time.sleep(3600)
                        all_tokens_checked = True

                continue
            return client.execute(
                query_template,
                variable_values={
                    "keyword": keyword,
                    "first": first,
                    "afterCursor": after_cursor,
                },
            )
        except Exception as e:
            if "API rate limit" in str(e):
                print(
                    f"Rate limit reached: {e}, switching token... (Attempt with first {first})"
                )
                current_token = next(token_iterator)
                headers["Authorization"] = f"Bearer {current_token}"
            else:
                if first > 1:
                    first = max(1, first // 2)
                    print(
                        f"Error: {e}, reducing number of results and retrying... (Attempt with first {first})"
                    )
                else:
                    break
    print("Max retries reached. Sleeping for 60 minutes and switching token...")
    time.sleep(3600)
    current_token = next(token_iterator)
    headers["Authorization"] = f"Bearer {current_token}"
    transport.headers = headers
    return execute_query(keyword, first, after_cursor)

In [219]:
import pickle

if os.path.exists("progress.pkl"):
    with open("progress.pkl", "rb") as f:
        progress_data = pickle.load(f)
        df = progress_data["df"]
        start_index = progress_data["start_index"]
else:
    df = []
    start_index = 0

In [220]:
df = []
start_index = 0

In [221]:
import datetime
from os import close

# template
keywords = ["spam"]
index = start_index
se_fm_repository_data = df
for keyword in tqdm(keywords):
    search_keyword = f"label:{keyword} created:2020-01-30..2024-01-30 is:pr is:public comments:>10 archived:false"
    try:
        after_cursor = None
        while True:
            response = execute_query(
                search_keyword, first=10, after_cursor=after_cursor
            )
            if response["search"]["issueCount"] == 0:
                break
            # Extract pr
            print("response", response)
            for edge in response["search"]["edges"]:
                pull_request = edge["node"]
                if not pull_request:
                    continue
                timeline = pull_request["timeline"]["edges"]
                labeled_spam_event = next(
                    filter(
                        lambda x: x["node"]
                        and x["node"]["__typename"] == "LabeledEvent"
                        and x["node"]["label"]["name"]
                        and (x["node"]["label"]["name"]).lower() == "spam",
                        timeline,
                    ),
                    None,
                )
                labeled_spam_event_node = (
                    labeled_spam_event["node"] if labeled_spam_event else None
                )
                closed_event = next(
                    filter(
                        lambda x: x["node"]
                        and x["node"]["__typename"] == "ClosedEvent",
                        timeline,
                    ),
                    None,
                )
                closed_event_node = closed_event["node"] if closed_event else None
                author = pull_request["author"]
                df.append(
                    {
                        "id": pull_request["id"],
                        "title": pull_request["title"],
                        "url": pull_request["url"],
                        "commits_count": pull_request["commits"]["totalCount"],
                        "changed_files_count": pull_request["changedFiles"],
                        "commits": pull_request["commits"]["edges"],
                        "labeled_spam_by": (
                            labeled_spam_event_node["actor"]["login"]
                            if labeled_spam_event_node
                            and labeled_spam_event_node["actor"]
                            else None
                        ),
                        "labeled_spam_at": (
                            labeled_spam_event_node["createdAt"]
                            if labeled_spam_event_node
                            else None
                        ),
                        "closed_by": (
                            closed_event_node["actor"]["login"]
                            if closed_event_node and closed_event_node["actor"]
                            else None
                        ),
                        "closed_at": (
                            closed_event_node["createdAt"]
                            if closed_event_node
                            else None
                        ),
                        "merged": pull_request["merged"],
                        "body": pull_request["body"],
                        "created_at": pull_request["createdAt"],
                        "updated_at": pull_request["updatedAt"],
                        "repository": pull_request["repository"],
                        "merge_commit": (
                            pull_request["mergeCommit"]["oid"]
                            if pull_request["mergeCommit"]
                            else None
                        ),
                        "labels": [
                            label["node"]["name"]
                            for label in pull_request["labels"]["edges"]
                        ],
                        "author_name": author['login'],
                        "author_url": author['url'],
                        "author_repository_count": author['repositories']['totalCount'],
                        "author_contributions_count": author['contributionsCollection']['contributionCalendar']['totalContributions'],
                        "author_followers_count": author["followers"]['totalCount'],
                        "author_following_count": author["following"]['totalCount'],
                        # "watchers": repo["watchers"]["totalCount"],
                        # "open_issues": repo["issues"]["totalCount"],
                        # "closed_issues": repo["issuesClosed"]["totalCount"],
                        # "total_issues": repo["issues"]["totalCount"]
                        # + repo["issuesClosed"]["totalCount"],
                        # "open_pull_requests": repo["pullRequests"]["totalCount"],
                        # "closed_pull_requests": repo["pullRequestsClosed"][
                        #     "totalCount"
                        # ],
                        # "total_pull_requests": repo["pullRequests"]["totalCount"]
                        # + repo["pullRequestsClosed"]["totalCount"],
                        # "commits": repo["defaultBranchRef"]["target"]["history"][
                        #     "totalCount"
                        # ],
                        # "last_commit": repo["pushedAt"],
                        # "releases": repo["releases"]["totalCount"],
                    }
                )

            # Pagination
            page_info = response["search"]["pageInfo"]
            if page_info["hasNextPage"]:
                after_cursor = page_info["endCursor"]
            else:
                break
        with open("progress.pkl", "wb") as f:
            pickle.dump({"df": se_fm_repository_data, "start_index": index + 1}, f)

    except Exception as e:
        print(f"Failed to retrieve data for keywords '{keyword}': {e}")
        # Save progress before terminating
        with open("progress.pkl", "wb") as f:
            pickle.dump({"df": df, "start_index": index}, f)
        raise

  0%|          | 0/1 [00:00<?, ?it/s]

Executing query with keyword: label:spam created:2020-01-30..2024-01-30 is:pr is:public comments:>10 archived:false, first: 10, afterCursor: None
response {'search': {'issueCount': 20, 'edges': [{'cursor': 'Y3Vyc29yOjE=', 'node': {'id': 'PR_kwDOAbI7X85jZl7M', 'number': 53006, 'title': 'fix: removed extra blank link between sections', 'url': 'https://github.com/freeCodeCamp/freeCodeCamp/pull/53006', 'body': 'Checklist:\r\n\r\n<!-- Please follow this checklist and put an x in each of the boxes, like this: [x]. It will ensure that our team takes your pull request seriously. -->\r\n\r\n- [ ] I have read and followed the [contribution guidelines](https://contribute.freecodecamp.org).\r\n- [ ] I have read and followed the [how to open a pull request guide](https://contribute.freecodecamp.org/#/how-to-open-a-pull-request).\r\n- [ ] My pull request targets the `main` branch of freeCodeCamp.\r\n- [ ] I have tested these changes either locally on my machine, or GitPod.\r\n\r\n<!--If your pull re

100%|██████████| 1/1 [00:05<00:00,  5.74s/it]

response {'search': {'issueCount': 20, 'edges': [{'cursor': 'Y3Vyc29yOjEx', 'node': {'id': 'PR_kwDOICigu85APCVM', 'number': 63, 'title': 'Updated suggest project', 'url': 'https://github.com/ImMattDavison/NoMoreDomains/pull/63', 'body': 'Hello, some contributions to the project. 🤗\r\n\r\n- Add .gitignore in project\r\n- Removed css in html\r\n- Add css in project\r\n- Updated Readme', 'state': 'CLOSED', 'merged': False, 'createdAt': '2022-10-05T18:15:43Z', 'updatedAt': '2022-10-05T20:04:17Z', 'mergeCommit': None, 'timeline': {'edges': [{'node': {'__typename': 'Commit'}}, {'node': {'__typename': 'Commit'}}, {'node': {'__typename': 'Commit'}}, {'node': {'__typename': 'Commit'}}, {'node': {'__typename': 'PullRequestReview'}}, {'node': {'__typename': 'LabeledEvent', 'actor': {'login': 'ImMattDavison'}, 'label': {'name': 'spam'}, 'createdAt': '2022-10-05T18:26:41Z'}}, {'node': {'__typename': 'RenamedTitleEvent'}}, {'node': {'__typename': 'LabeledEvent', 'actor': {'login': 'ImMattDavison'}, 

In [222]:
import pickle
import pandas as pd


def display_pkl_content(filepath):
    """
    Loads and displays the content of a .pkl (pickle) file in a Jupyter Notebook.
    Handles different data structures within the pickle file and provides informative output.
    """
    try:
        with open(filepath, "rb") as f:
            data = pickle.load(f)

        print(f"Content of {filepath}:\n")

        if isinstance(data, pd.DataFrame):
            print("Data is a Pandas DataFrame:")
            print(
                data.head().to_markdown(index=False, numalign="left", stralign="left")
            )  # Display first few rows as a Markdown table
            print("\nDataFrame Info:")
            data.info()  # Display DataFrame information
        elif isinstance(data, list):
            print("Data is a List:")
            if all(
                isinstance(item, dict) for item in data
            ):  # Check if list contains dictionaries
                df = pd.DataFrame(data)
                print(
                    df.head().to_markdown(index=False, numalign="left", stralign="left")
                )  # Display first few rows as a Markdown table
                print("\nDataFrame Info:")
                df.info()  # Display DataFrame information
            else:
                print(data[:10])  # Print first 10 items if not dictionaries
                print(f"\nList Length: {len(data)}")
        elif isinstance(data, dict):
            print("Data is a Dictionary:")
            pd.DataFrame(data["df"]).to_csv("data_set.csv", index=False)
            for key, value in data.items():
                print(f"\nKey: {key}")
                if isinstance(value, pd.DataFrame):
                    print("Value is a Pandas DataFrame:")
                    print(
                        value.head().to_markdown(
                            index=False, numalign="left", stralign="left"
                        )
                    )
                    print("\nDataFrame Info:")
                    value.info()
                elif isinstance(value, list):
                    print("Value is a List:")
                    print(value[:10])
                    print(f"\nList Length: {len(value)}")

                else:
                    print(f"Value: {value}")
        elif isinstance(data, set):
            print("Data is a Set:")
            print(list(data)[:10])
            print(f"\nSet Length: {len(data)}")

        else:
            print(f"Data is of type: {type(data)}")
            print(data)

    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
    except pickle.UnpicklingError:
        print(
            f"Error: Could not unpickle data from {filepath}. The file might be corrupted or use a different pickle protocol."
        )
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Example usage:
filepath = "progress.pkl"  # Replace with the actual path to your .pkl file
display_pkl_content(filepath)

Content of progress.pkl:

Data is a Dictionary:

Key: df
Value is a List:
[{'id': 'PR_kwDOAbI7X85jZl7M', 'title': 'fix: removed extra blank link between sections', 'url': 'https://github.com/freeCodeCamp/freeCodeCamp/pull/53006', 'commits_count': 1, 'changed_files_count': 1, 'commits': [{'node': {'commit': {'oid': '9deca889c27f0fc48b2bbe4f9505277b3c352572', 'message': 'fix: removed extra blank link between sections', 'author': {'name': 'Surya Tejess Karri'}, 'changedFilesIfAvailable': 1, 'commitUrl': 'https://github.com/freeCodeCamp/freeCodeCamp/commit/9deca889c27f0fc48b2bbe4f9505277b3c352572', 'committedDate': '2024-01-07T06:45:31Z'}}}], 'labeled_spam_by': None, 'labeled_spam_at': '2024-01-07T06:46:17Z', 'closed_by': None, 'closed_at': '2024-01-07T06:46:16Z', 'merged': False, 'body': 'Checklist:\r\n\r\n<!-- Please follow this checklist and put an x in each of the boxes, like this: [x]. It will ensure that our team takes your pull request seriously. -->\r\n\r\n- [ ] I have read and fol